## UVA MSDS Capstone Project Data Processing Pipeline Step 1
- This notebook uses R syntax to read 150NT QZA files and export them as .csv files for ease of use with python
- It also reads in the Silva Taxonomy file and save it as a .csv file

### Loading Bioinformatics R Packages
- The chunk below installs the required R packages including "dada2" and "qiime2R"

In [2]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(version = "3.15")
BiocManager::install(c("dada2"))

if (!requireNamespace("devtools", quietly = TRUE)){install.packages("devtools")}
devtools::install_github("jbisanz/qiime2R")

library("dada2")
library("yaml")
library("qiime2R")

Warning message:
"package 'BiocManager' was built under R version 4.2.1"
'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.0 (2022-04-22 ucrt)

Old packages: 'aplot', 'broom', 'bslib', 'callr', 'car', 'caret', 'classInt',
  'cli', 'cluster', 'DBI', 'dbplyr', 'dendextend', 'desc', 'devtools', 'dials',
  'dplyr', 'DT', 'dtplyr', 'FactoMineR', 'fontawesome', 'forcats', 'furrr',
  'future', 'future.apply', 'gam', 'gargle', 'gbm', 'GenomeInfoDb', 'gert',
  'ggfun', 'gh', 'gitcreds', 'globals', 'googlesheets4', 'gtable', 'gtools',
  'hardhat', 'haven', 'hms', 'htmltools', 'httpuv', 'httr', 'infer', 'kernlab',
  'klaR', 'labelled', 'lifecycle', 'lme4', 'lobstr', 'MASS', 'Matrix',
  'MatrixModels', 'modeldata', 'modelr', 'multcomp', 'nlme', 'openssl', 'ottr',
  'paletteer', 'parallelly', 'parsnip', 'patchwork', 'pkgload', 'pls'

## Setting Up Directories and Reading in Sample CSV

In [9]:
## This notebook sits in the following working directory (code may need to be tweaked to fit your own directory)
getwd()

[1] "C:/Users/Capstone1/Desktop/data_processing"

In [10]:
## Displaying all of the processed feature tables in .qza format that will be converted
list.files(file.path(getwd(), "150NT_files"))

[1] "127044_feature-table.qza" "127158_feature-table.qza"
 [3] "127159_feature-table.qza" "127271_feature-table.qza"
 [5] "127292_feature-table.qza" "127307_feature-table.qza"
 [7] "127419_feature-table.qza" "128190_feature-table.qza"
 [9] "128225_feature-table.qza" "128246_feature-table.qza"
[11] "128291_feature-table.qza" "128310_feature-table.qza"
[13] "128319_feature-table.qza" "128433_feature-table.qza"
[15] "128483_feature-table.qza" "128555_feature-table.qza"
[17] "128622_feature-table.qza" "128663_feature-table.qza"
[19] "128669_feature-table.qza" "128705_feature-table.qza"
[21] "128764_feature-table.qza" "128788_feature-table.qza"
[23] "128801_feature-table.qza" "128817_feature-table.qza"
[25] "128961_feature-table.qza" "129003_feature-table.qza"
[27] "129020_feature-table.qza" "129022_feature-table.qza"
[29] "129074_feature-table.qza" "129085_feature-table.qza"
[31] "129154_feature-table.qza" "129191_feature-table.qza"
[33] "129318_feature-table.qza" "129435_feature-table.qza"
[35] "129441_feature-table.qza" "129487_feature-table.qza"
[37] "129729_feature-table.qza" "129741_feature-table.qza"
[39] "130045_feature-table.qza" "130065_feature-table.qza"
[41] "130091_feature-table.qza" "130252_feature-table.qza"
[43] "130336_feature-table.qza" "130532_feature-table.qza"
[45] "130579_feature-table.qza" "130691_feature-table.qza"
[47] "131011_feature-table.qza" "131135_feature-table.qza"
[49] "131162_feature-table.qza" "131201_feature-table.qza"
[51] "131417_feature-table.qza" "131423_feature-table.qza"
[53] "131433_feature-table.qza" "131451_feature-table.qza"
[55] "131459_feature-table.qza" "131495_feature-table.qza"
[57] "131583_feature-table.qza" "131755_feature-table.qza"
[59] "131762_feature-table.qza" "131775_feature-table.qza"
[61] "131792_feature-table.qza" "132036_feature-table.qza"
[63] "132107_feature-table.qza" "132133_feature-table.qza"
[65] "132142_feature-table.qza" "132153_feature-table.qza"
[67] "132366_feature-table.qza" "132390_feature-table.qza"
[69] "132546_feature-table.qza" "132773_feature-table.qza"
[71] "133199_feature-table.qza" "133335_feature-table.qza"
[73] "133650_feature-table.qza" "133716_feature-table.qza"
[75] "133816_feature-table.qza" "133907_feature-table.qza"
[77] "134003_feature-table.qza" "134139_feature-table.qza"
[79] "134191_feature-table.qza" "134562_feature-table.qza"
[81] "134836_feature-table.qza" "134925_feature-table.qza"
[83] "135404_feature-table.qza" "135460_feature-table.qza"
[85] "135637_feature-table.qza" "135983_feature-table.qza"
[87] "135988_feature-table.qza" "135999_feature-table.qza"
[89] "136107_feature-table.qza" "139281_feature-table.qza"
[91] "139314_feature-table.qza" "142215_feature-table.qza"
[93] "146212_feature-table.qza" "147022_feature-table.qza"
[95] "153674_feature-table.qza"

In [12]:
## Loading the sample survery information file
info <- read.csv("metadata_files/sample_info.csv")
head(info)

,bmi_cat,bowel_movement_frequency,bowel_movement_quality,pregnant,race,sample_type,sample_name,sex,vegetable_frequency,kidney_disease,⋯,cdiff,liver_disease,lung_disease,diet_type,whole_grain_frequency,meat_eggs_frequency,milk_cheese_frequency,prepared_meals_frequency,age_cat,diagnosis
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Normal,Three,,No,Caucasian,feces,10317.000001792,female,Daily,"Diagnosed by a medical professional (doctor, physician assistant)",⋯,I do not have this condition,I do not have this condition,"Diagnosed by a medical professional (doctor, physician assistant)",Omnivore,Never,Daily,Never,Never,50s,kidney_disease
2,Obese,Two,I tend to have normal formed stool - Type 3 and 4,No,Caucasian,feces,10317.000003540,female,Regularly (3-5 times/week),"Diagnosed by a medical professional (doctor, physician assistant)",⋯,"Diagnosed by a medical professional (doctor, physician assistant)",I do not have this condition,"Diagnosed by a medical professional (doctor, physician assistant)",Omnivore,Never,Regularly (3-5 times/week),Rarely (less than once/week),Rarely (less than once/week),50s,kidney_disease
3,Overweight,One,I tend to be constipated (have difficulty passing stool) - Type 1 and 2,No,Caucasian,feces,10317.000003560,female,Daily,"Diagnosed by a medical professional (doctor, physician assistant)",⋯,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Rarely (less than once/week),Daily,Occasionally (1-2 times/week),Occasionally (1-2 times/week),40s,kidney_disease
4,Normal,Two,I tend to have normal formed stool - Type 3 and 4,No,Caucasian,feces,10317.000007157,male,Daily,"Diagnosed by a medical professional (doctor, physician assistant)",⋯,,I do not have this condition,"Diagnosed by a medical professional (doctor, physician assistant)",Omnivore,Rarely (less than once/week),Daily,Occasionally (1-2 times/week),Rarely (less than once/week),60s,kidney_disease
5,Normal,Two,I tend to have normal formed stool - Type 3 and 4,No,Caucasian,feces,10317.000010099,male,Regularly (3-5 times/week),"Diagnosed by a medical professional (doctor, physician assistant)",⋯,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Occasionally (1-2 times/week),Daily,Regularly (3-5 times/week),Regularly (3-5 times/week),40s,kidney_disease
6,Underweight,Less than one,I tend to be constipated (have difficulty passing stool) - Type 1 and 2,No,Caucasian,feces,10317.000012046,male,Occasionally (1-2 times/week),"Diagnosed by a medical professional (doctor, physician assistant)",⋯,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Occasionally (1-2 times/week),Regularly (3-5 times/week),Rarely (less than once/week),Rarely (less than once/week),40s,kidney_disease


## Getting the Taxonomy Assignments and writing to CSV

In [27]:
### This tab takes a long time, just refer to the csv created in metadata_files from it

### Assigning Taxonomy to ~70k sequenes
df <- read_qza("150NT_files/130252_feature-table.qza")$data
seqs = as.data.frame(rownames(df))
seqs$abundance = 1
colnames(seqs) = c("sequence", "abundance")
taxa <- assignTaxonomy(seqs, "metadata_files/silva_nr_v132_train_set.fa.gz", multithread=TRUE)

In [28]:
### Displaying 5 of the sequences with taxonomy applied
head(taxa)

,Kingdom,Phylum,Class,Order,Family,Genus
TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCGCGTAGGCGGTTTTTTAAGTCTGATGTGAAAGCCCACGGCTCAACCGTGGAGGGTCATTGGAAACTGGAAAACTTGAGTGCAGAAGAGGAAAGTGGAATTCCATG,Bacteria,Firmicutes,Bacilli,Bacillales,Staphylococcaceae,Staphylococcus
TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCATAACTTGAGTACTGTAGGGGTAACTGGAATTCCTG,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium_1
TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTGTGATAGAGGGTGGTAGAATTTCAGG,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Pseudoalteromonadaceae,Pseudoalteromonas
TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTTTGTGTAAGCCCGCAGCTTAACTGCGGGACTGCAGGCGATACGGGCATAACTTGAGTGCTGTAGGGGAGACTGGAATTCCTG,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium_1
TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCACGTCGTCTGTGAAATCCTAGGGCTTAACCCTGGACGTGCAGGCGATACGGGCTGACTTGAGTACTACAGGGGAGACTGGAATTTCTGG,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Lawsonella
TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATCCCGGGGCTTAACTTCGGGCGTGCAGGCGATACGGGCATAACTAGAGTGCTGTAGGGGAGACTGGAATTCCTG,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium_1


In [29]:
### Writing taxa information to metadata files folder to avoid running the above code again
write.csv(taxa, "metadata_files/big_tax_table.csv")

### Reading all .qza files and writing them as .csvs and storing them in a folder

In [41]:
list.files(file.path(getwd(), "150NT_files"))

[1] "127044_feature-table.qza" "127158_feature-table.qza"
 [3] "127159_feature-table.qza" "127271_feature-table.qza"
 [5] "127292_feature-table.qza" "127307_feature-table.qza"
 [7] "127419_feature-table.qza" "128190_feature-table.qza"
 [9] "128225_feature-table.qza" "128246_feature-table.qza"
[11] "128291_feature-table.qza" "128310_feature-table.qza"
[13] "128319_feature-table.qza" "128433_feature-table.qza"
[15] "128483_feature-table.qza" "128555_feature-table.qza"
[17] "128622_feature-table.qza" "128663_feature-table.qza"
[19] "128669_feature-table.qza" "128705_feature-table.qza"
[21] "128764_feature-table.qza" "128788_feature-table.qza"
[23] "128801_feature-table.qza" "128817_feature-table.qza"
[25] "128961_feature-table.qza" "129003_feature-table.qza"
[27] "129020_feature-table.qza" "129022_feature-table.qza"
[29] "129074_feature-table.qza" "129085_feature-table.qza"
[31] "129154_feature-table.qza" "129191_feature-table.qza"
[33] "129318_feature-table.qza" "129435_feature-table.qza"
[35] "129441_feature-table.qza" "129487_feature-table.qza"
[37] "129729_feature-table.qza" "129741_feature-table.qza"
[39] "130045_feature-table.qza" "130065_feature-table.qza"
[41] "130091_feature-table.qza" "130252_feature-table.qza"
[43] "130336_feature-table.qza" "130532_feature-table.qza"
[45] "130579_feature-table.qza" "130691_feature-table.qza"
[47] "131011_feature-table.qza" "131135_feature-table.qza"
[49] "131162_feature-table.qza" "131201_feature-table.qza"
[51] "131417_feature-table.qza" "131423_feature-table.qza"
[53] "131433_feature-table.qza" "131451_feature-table.qza"
[55] "131459_feature-table.qza" "131495_feature-table.qza"
[57] "131583_feature-table.qza" "131755_feature-table.qza"
[59] "131762_feature-table.qza" "131775_feature-table.qza"
[61] "131792_feature-table.qza" "132036_feature-table.qza"
[63] "132107_feature-table.qza" "132133_feature-table.qza"
[65] "132142_feature-table.qza" "132153_feature-table.qza"
[67] "132366_feature-table.qza" "132390_feature-table.qza"
[69] "132546_feature-table.qza" "132773_feature-table.qza"
[71] "133199_feature-table.qza" "133335_feature-table.qza"
[73] "133650_feature-table.qza" "133716_feature-table.qza"
[75] "133816_feature-table.qza" "133907_feature-table.qza"
[77] "134003_feature-table.qza" "134139_feature-table.qza"
[79] "134191_feature-table.qza" "134562_feature-table.qza"
[81] "134836_feature-table.qza" "134925_feature-table.qza"
[83] "135404_feature-table.qza" "135460_feature-table.qza"
[85] "135637_feature-table.qza" "135983_feature-table.qza"
[87] "135988_feature-table.qza" "135999_feature-table.qza"
[89] "136107_feature-table.qza" "139281_feature-table.qza"
[91] "139314_feature-table.qza" "142215_feature-table.qza"
[93] "146212_feature-table.qza" "147022_feature-table.qza"
[95] "153674_feature-table.qza"

In [47]:
## Looping through all of the qzas and converting to .csvs
for(i in 1:length(list.files(file.path(getwd(), "150NT_files")))) {
    ## Read the file
    df <- as.data.frame(read_qza(paste("150NT_files",list.files(file.path(getwd(), "150NT_files"))[i], sep = "/"))$data)
    ## Write the file to csv
    write.csv(df, paste("150NT_csvs",paste(substr(list.files(file.path(getwd(), "150NT_files"))[i],1,6), "csv", sep = "."), sep = "\\"))
}

In [1]:
list.files(file.path(getwd(), "150NT_csvs"))

[1] "127044.csv" "127158.csv" "127159.csv" "127271.csv" "127292.csv"
 [6] "127307.csv" "127419.csv" "128190.csv" "128225.csv" "128246.csv"
[11] "128291.csv" "128310.csv" "128319.csv" "128433.csv" "128483.csv"
[16] "128555.csv" "128622.csv" "128663.csv" "128669.csv" "128705.csv"
[21] "128764.csv" "128788.csv" "128801.csv" "128817.csv" "128961.csv"
[26] "129003.csv" "129020.csv" "129022.csv" "129074.csv" "129085.csv"
[31] "129154.csv" "129191.csv" "129318.csv" "129435.csv" "129441.csv"
[36] "129487.csv" "129729.csv" "129741.csv" "130045.csv" "130065.csv"
[41] "130091.csv" "130252.csv" "130336.csv" "130532.csv" "130579.csv"
[46] "130691.csv" "131011.csv" "131135.csv" "131162.csv" "131201.csv"
[51] "131417.csv" "131423.csv" "131433.csv" "131451.csv" "131459.csv"
[56] "131495.csv" "131583.csv" "131755.csv" "131762.csv" "131775.csv"
[61] "131792.csv" "132036.csv" "132107.csv" "132133.csv" "132142.csv"
[66] "132153.csv" "132366.csv" "132390.csv" "132546.csv" "132773.csv"
[71] "133199.csv" "133335.csv" "133650.csv" "133716.csv" "133816.csv"
[76] "133907.csv" "134003.csv" "134139.csv" "134191.csv" "134562.csv"
[81] "134836.csv" "134925.csv" "135404.csv" "135460.csv" "135637.csv"
[86] "135983.csv" "135988.csv" "135999.csv" "136107.csv" "139281.csv"
[91] "139314.csv" "142215.csv" "146212.csv" "147022.csv" "153674.csv"

## Summary of notebook outputs
- 150NT_csvs folder: contains all the feature tables as .csvs
- meta_data folder: contains the sample survey information and the taxonomy table
- This notebook is written in R syntax, the other notebooks in this pipeline utilize Python for analysis